In [134]:
import numpy as np
import rocketcea 
import pint
import time
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
pint.__version__  
from pint import UnitRegistry

from rocketcea.cea_obj import CEA_Obj
from functools import cached_property

In [135]:
#Generating CEA object and exit velocity (67% overall efficiency)
C = CEA_Obj(oxName="LOX", fuelName="Kerosene")
V_exit = C.get_Isp(Pc=356, MR=1.5, eps=5.2523, frozen=1) * 0.67 * 9.81

#Calculate maximum possible mass flow/thrust based on ALI CdA and NF Setpoint
P_c=356 #psia
OF=1.5
CdA_combined_LOX = 1.884E-04 #ft^2
CdA_combined_KERO = 9.074E-05 #ft^2

#Assume LOX at 90 K, KERO at room temp
rho_LOX = 2.222 #slugs/ft^3
rho_KERO = 1.591 #slugs/ft^3

mdot_max_LOX = CdA_combined_LOX*np.sqrt(2*rho_LOX*144*(1000-P_c)) * 14.5939029
mdot_max_KERO = CdA_combined_KERO*np.sqrt(2*rho_KERO*144*(1000-P_c)) * 14.5939029 

print(f"Maximum LOX Massflow: {mdot_max_LOX} kg/s")
print(f"Corresponding KERO Massflow: {mdot_max_LOX/OF} kg/s \n")

print(f"Maximum KERO Massflow: {mdot_max_KERO} kg/s")
print(f"Corresponding LOX Massflow: {mdot_max_KERO*OF} kg/s")

Maximum LOX Massflow: 1.765076592819631 kg/s
Corresponding KERO Massflow: 1.1767177285464208 kg/s 

Maximum KERO Massflow: 0.7193570341772857 kg/s
Corresponding LOX Massflow: 1.0790355512659286 kg/s


As seen in the above results, the maximum total massflow is limited by the kerosene line, as the corresponding kerosene massflow for case 1 is greater than the maximum massflow as seen in case 2. 

In [136]:
#Initializaing Core Variables and Lists
mdot_max_TOTAL = mdot_max_KERO + mdot_max_KERO*OF

mdot_arr = np.linspace(0, mdot_max_TOTAL, num=51)
thrust_arr = V_exit*mdot_arr
tburn_arr = np.linspace(20, 0, num=51)

m_dry = 55 #kg
rho_ATM = 1.225 #kg/m^3

#Initializing altitude
apogee_altitude = [[0 for x in thrust_arr] for y in tburn_arr]

In [137]:
#Integrator Function Definition

def altitude_integrator(thrust, tburn, m_wet, mdot, dt, drag=True):
    #Initialize Lists and Variables
    x_arr = []
    t_arr = []
    v = 0
    x = 0
    t = 0
    m = m_wet

    #Aerodynamic Variables
    Cf = 0
    Cf_fins = 0
    mu_ATM = 1.789E-05 #Pa*s

    #For Body/Nosecone
    L = 4.41 #m
    A_wet = 2.166 + 0.299542230065 #m^2 (body contribution + nosecone contribution)
    R_s = 2.0E-05 #m
    R_d_crit = 51*(R_s/L)**(-1.039)

    #For Fins
    L_fins = 0.305 #m
    A_wet_fins = 0.28512 #m^2 (contribution of 4 fins)
    R_s_fins = 2.0E-05 #m
    R_d_crit_fins = 51*(R_s/L)**(-1.039)

    while v >=0:
            #Sets thrust and massflow to 0 at end of burn
            if t > tburn:
                thrust = 0
                mdot = 0
            
            #Update mass
            m = m-(mdot*dt) #kg

            ### BODY/NOSECONE SKIN FRICTION DRAG CALCULATION ###\

            if drag:
                R_d = (rho_ATM * v * L)/(mu_ATM)
                M = v/343 #m/s

                #Reynolds Number Regimes
                if R_d < 1.0E4:
                    Cf = 1.48E-02
                elif (1.0E4 <= R_d) & (R_d <= R_d_crit):
                    Cf = 1/((1.5*np.log(R_d)-5.6)**2)
                elif R_d_crit < R_d:
                    Cf = 0.032*(R_s/L)**0.2
                else:
                    Cf = 0

                #Mach Number Regimes
                if M < 1:
                    Cfc = Cf*(1 - 0.1*M**2)
                elif (1 <= M) & (R_d < R_d_crit):
                    Cfc = Cf/((1+0.15*M**2)**0.58)
                elif (1 <= M) & (R_d > R_d_crit):
                    Cfc = Cf/(1+0.18*M**2)

                D = Cfc*0.5*rho_ATM*(v**2)*A_wet
            else:
                D=0

            ### FINS SKIN FRICTION DRAG CALCULATION ###

            if drag:
                R_d_fins = (rho_ATM * v * L_fins)/(mu_ATM)

                #Reynolds Number Regimes
                if R_d_fins < 1.0E4:
                    Cf_fins = 1.48E-02
                elif (1.0E4 <= R_d_fins) & (R_d_fins <= R_d_crit_fins):
                    Cf_fins = 1/((1.5*np.log(R_d_fins)-5.6)**2)
                elif R_d_crit_fins < R_d_fins:
                    Cf_fins = 0.032*(R_s_fins/L_fins)**0.2
                else:
                    Cf_fins = 0

                #Mach Number Regimes
                if M < 1:
                    Cfc_fins = Cf_fins*(1 - 0.1*M**2)
                elif (1 <= M) & (R_d_fins < R_d_crit_fins):
                    Cfc_fins = Cf_fins/((1+0.15*M**2)**0.58)
                elif (1 <= M) & (R_d_fins > R_d_crit_fins):
                    Cfc_fins = Cf_fins/(1+0.18*M**2)

                D_fins = Cfc_fins*0.5*rho_ATM*(v**2)*A_wet_fins
            else:
                D_fins = 0

            #Update velocity
            v_old = v
            dv = ((thrust/m) - 9.81 - (D/m) - (D_fins/m))*dt
            v = v + dv

            #Update position
            dx = ((v+v_old)/2)*dt
            x = x + dx
            t = t + dt

            x_arr.append(x)
            t_arr.append(t)

    #print(f"Altitude at Apogee: {x}")
    return(x, x_arr, t_arr)

In [138]:
#Running the Full Estimator
dt = 0.01

#Loop through entire matrix
for i, thrust in enumerate(thrust_arr):
    for j, tburn in enumerate(tburn_arr):
        mdot = mdot_arr[i]
        m_prop_total = mdot * tburn
        m_wet = m_dry + m_prop_total

        x, x_arr, t_arr = altitude_integrator(thrust = thrust, tburn = tburn, m_wet = m_wet, mdot = mdot, dt = dt)
        
        apogee_altitude[j][i] = x
        #print(f"Thrust:{thrust} | Burn Time:{tburn} | Altitude:{x}")

In [139]:
#Plotting

#Heat Map
fig1 = px.imshow(apogee_altitude, 
                labels=dict(x = "Thrust (N)",
                            y = "Burn Time (s)",
                            color = "Altitude (m)"),
                            x = thrust_arr,
                            y = tburn_arr,
                            aspect = "auto",
                            origin = "lower",
                            title="Apogee Altitude vs Thrust & Burn Time",
                            )
fig1.update_layout(
        width=800,
        height=700
    )
fig1.show()

#Surface Map
fig2 = go.Figure(data = go.Surface(z = apogee_altitude,
                                x = thrust_arr,
                                y = tburn_arr,
                                contours = {"x": {"show": False, "start": 889.6297399465218, "end": 3199.8070562191997, "size": (3199.8070562191997-889.6297399465218)/20},
                                                           "y": {"show": False, "start": 5, "end": 20, "size": (20-5)/21},
                                                           "z": {"show": True, "start": 0, "end": 15000, "size": 1000}},
                                ))

fig2.update_layout(
        title="Apogee Altitude vs Thrust & Burn Time",
        scene = dict(
                      xaxis=dict(
                          title=dict(
                              text="Thrust (N)"
                          )
                      ),
                      yaxis=dict(
                          title=dict(
                              text="Burn Time (s)"
                          )
                      ),
                      zaxis=dict(
                          title=dict(
                              text="Apogee Altitude (m)"
                          )
                      ),
                    ),
        width=800,
        height=700,
    )

fig2.update_scenes(xaxis_autorange="reversed")
fig2.update_scenes(yaxis_autorange="reversed")


fig2.show()

In [140]:
#Example altitude_integrator() run
x, x_arr, t_arr = altitude_integrator(thrust = 2157, tburn = 10, m_wet = 67.3184496663, mdot = 1.2918, dt = 0.001)
x_dragless, x_arr_dragless, t_arr_dragless = altitude_integrator(thrust = 2157, tburn = 10, m_wet = 67.3184496663, mdot = 1.2918, dt = 0.001, drag = False)

fig3 = go.Figure(
    layout = go.Layout(
        title=go.layout.Title(text="Vehicle Altitude vs Time | Thrust = 2157 N | Burn Time = 10 s"),
        width = 800,
        height = 700
    )
)
fig3.add_trace(go.Scatter(x = t_arr, y = x_arr, mode = "lines", name = "With Drag"))
fig3.add_trace(go.Scatter(x = t_arr_dragless, y = x_arr_dragless, mode="lines", name="Without Drag"))

fig3.update_layout(
    xaxis_title = "Time Since Launch (s)",
    yaxis_title = "Altitude (m)"
)

fig3.show()

In [141]:
plot_div1 = fig1.to_html(full_html=False, include_plotlyjs='cdn')
plot_div2 = fig2.to_html(full_html=False, include_plotlyjs='cdn')
plot_div3 = fig3.to_html(full_html=False, include_plotlyjs='cdn')

with open("Altitude Estimator V1 Output.html", "a") as f:
    f.truncate(0)
    f.write(plot_div1)
    f.write(plot_div2)
    f.write(plot_div3)
